# setup

In [1]:
base_url="http://host.docker.internal:11434"
model = "gemma3:27b"
from langchain_ollama import ChatOllama
model = ChatOllama(
    model=model,
    base_url=base_url,
)

# tools

In [2]:
import inspect
from functools import wraps
from pydantic import create_model

def register_function_schema(func):
    sig = inspect.signature(func)

    # Build Pydantic model dynamically from function arguments
    fields = {
        name: (param.annotation, ...)
        for name, param in sig.arguments.items()
    }

    ParamsModel = create_model(f"{func.__name__.title()}Params", **fields)

    # Attach schema to the function
    func._schema = {
        "name": func.__name__,
        "description": func.__doc__,
        "arguments": ParamsModel.model_json_schema()
    }

    return func

In [3]:
from typing import Any, Dict, Optional, TypedDict
from langchain_core.runnables import RunnableConfig

class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

In [26]:
from langchain_core.tools import tool


@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers together."""
    return a * b


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y

tools = [multiply, add]


# @register_function_schema
# def multiply(x: float, y: float) -> float:
#     """Multiply two numbers together."""
#     return x * y


# @register_function_schema
# def add(x: int, y: int) -> int:
#     "Add two numbers."
#     return x + y

# r = json.dumps([tool._schema for tool in tools], indent=4)


# from pydantic import BaseModel

# class UserInput(BaseModel):
#     name: str
#     age: int

# def process_user(data: UserInput):
#     return f"{data.name} is {data.age} years old."

# # Example usage
# user = UserInput(name="Alice", age=30)
# print(process_user(user))
# UserInput.model_json_schema()

# from pydantic import BaseModel

# class UserInput(BaseModel):
#     name: str
#     age: int

# def process_user(data: UserInput):
#     return f"{data.name} is {data.age} years old."

# # Example usage
# user = UserInput(name="Alice", age=30)
# print(process_user(user))
# UserInput.model_json_schema()

invoke_tool({"name": "multiply", "arguments": {"a": 3, "b": 5}})

15.0

In [5]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(a: float, b: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.


# single tool calling at a time

In [6]:
system_prompt = f"""\
You have access to functions. If you decide to invoke any of the function(s),
you MUST put it in the format of
{{{{"name": function name, "arguments": dictionary of argument name and its value}}}}

You SHOULD NOT include any other text in the response if you call a function
{rendered_tools}

"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [7]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4 and 12 plus 7"})

{'name': 'multiply', 'arguments': {'a': 13, 'b': 4}}

In [8]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply',
 'arguments': {'a': 13, 'b': 4.14137281},
 'output': 53.83784653}

# multi-tool calling

In [9]:
# [
#   {{{{
#     "name": "tool_name",
#     "arguments": {{{{
#       "x": value1,
#       "y": value2
#     }}}}
#   }}}},
#   ...
# ]

# [{{{{"name": function name, "arguments": }}}}, {{{{...}}}}]

In [10]:
system_prompt = f"""\
You have access to functions. If you decide to invoke any of the function(s),
you MUST put it in the format of

[
  {{{{
    "name": "tool_name",
    "arguments": dictionary of argument name and its value
  }}}},
  ...
]

You SHOULD NOT include any other text in the response if you call a function
{rendered_tools}
"""

print(system_prompt)
prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

You have access to functions. If you decide to invoke any of the function(s),
you MUST put it in the format of

[
  {{
    "name": "tool_name",
    "arguments": dictionary of argument name and its value
  }},
  ...
]

You SHOULD NOT include any other text in the response if you call a function
multiply(a: float, b: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.



In [19]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4 and 12 plus 7"})

[{'name': 'multiply', 'arguments': {'a': 13, 'b': 4}},
 {'name': 'add', 'arguments': {'x': 12, 'y': 7}}]

In [20]:
def run_tool_calls(tool_calls: list[ToolCallRequest]):
    results = []
    for call in tool_calls:
        result = invoke_tool(call)
        results.append({
            "name": call["name"],
            "arguments": call["arguments"],
            "output": result
        })
    return results

In [24]:
from langchain_core.runnables import RunnableLambda

chain = prompt | model | JsonOutputParser() | RunnableLambda(run_tool_calls)

In [25]:
full_chain.invoke({"input": "what is 234 * 3 and what is 293 + 98"})

[{'name': 'multiply', 'arguments': {'a': 234, 'b': 3}, 'output': 702.0},
 {'name': 'add', 'arguments': {'x': 293, 'y': 98}, 'output': 391}]

# add summarizer

In [34]:
from langchain_core.prompts import ChatPromptTemplate

summarize_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant. Summarize the tool results in simple, natural language. "
        "Avoid verbose introductions. Just clearly state what was computed."
    ),
    (
        "user",
        "Tool results:\n{results}"
    )
])

summarize_chain = summarize_prompt | model

In [35]:
from langchain_core.runnables import RunnableLambda

# The full chain: parse → invoke → summarize
chain = (
    prompt
    | model
    | JsonOutputParser()
    | RunnableLambda(run_tool_calls)
    | RunnableLambda(lambda state: summarize_chain.invoke({"results": state}))
    | RunnableLambda(lambda x: x.content if hasattr(x, "content") else x)
)


In [39]:
chain.invoke({"input": "what is 234 * 3 and 293 + 98"})

'234 multiplied by 3 equals 702. \n\n293 plus 98 equals 391.\n\n\n\n'